### En este Notebook definimos el alcance del proyecto eligiendo que categorias vamos a usar, para llegar a la conclusion de las categorias puede revisar el Notebook EDA/eda.ipynb donde con procesos de ML se normalizaron las categorias para poder filtrar aqui el alcance por estados y categorias mas relevantes del dataset.

In [2]:
import pandas as pd

In [9]:
# Contamos la cantidad de restaurantes y hoteles que contienen los estados, para elegir un Top 5 de mayor cantidad de negocios 
conteo_negocios_por_estado = g_sitios.groupby('id_estado').size().reset_index(name='cantidad_negocios')
conteo_negocios_por_estado.sort_values(by='cantidad_negocios', ascending=False)

,id_estado,cantidad_negocios
1,2,5140
4,5,3715
5,6,3576
12,13,2993
11,12,1952
13,14,1566
21,22,1419
19,20,1360
9,10,1341
0,1,1323


In [10]:
# Calcular el total de negocios
total_negocios = conteo_negocios_por_estado['cantidad_negocios'].sum()

# Calcular el porcentaje del total para cada id_estado
conteo_negocios_por_estado['porcentaje'] = round((conteo_negocios_por_estado['cantidad_negocios'] / total_negocios) * 100,2)
exportar_info = conteo_negocios_por_estado.copy() 
# Mostrar el resultado
conteo_negocios_por_estado = conteo_negocios_por_estado.sort_values(by='porcentaje', ascending=False).head(5)

- 2 = California
- 5 = Florida
- 6 = Texas
- 13 = New York
- 12 = Pennsylvania

Ahora que achicamos la informacion en la tabla principal. tambien debemos achicar la informacion de las demas tablas (address, horarios, misc, relative_results). 

In [6]:
# Leemos el archivo de google que tenemos hasta el momento.
g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\google_sitios_v2.parquet', engine='pyarrow')
google_estados = [2,5,6,13,12]
g_sitios = g_sitios[g_sitios['id_estado'].isin(google_estados)]
g_sitios.reset_index()
g_sitios.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17376 entries, 0 to 46518
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   name                 17376 non-null  object 
 1   id_negocio           17376 non-null  object 
 2   description          1246 non-null   object 
 3   avg_rating           17376 non-null  float64
 4   num_of_reviews       17376 non-null  int64  
 5   price                954 non-null    object 
 6   url                  17376 non-null  object 
 7   id_estado            17376 non-null  int32  
 8   id_category_general  17376 non-null  float64
dtypes: float64(2), int32(1), int64(1), object(5)
memory usage: 1.3+ MB


In [7]:
# Leemos los archivos que tenemos hasta el momento.
g_horarios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_horarios_v2.parquet', engine='pyarrow')
g_misc = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_misc_v3.parquet', engine='pyarrow')
g_address = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_address_v2.parquet', engine='pyarrow')
g_relative_results = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_relative_results_v2.parquet', engine='pyarrow')

In [ ]:
# Renombramos los id para mayor intuitividad. 
g_relative_results.rename(columns={'gmap_id': 'id_negocio'}, inplace=True)
g_misc.rename(columns={'gmap_id': 'id_negocio'}, inplace=True)

In [8]:
# Obtenemos los ID unicos
g_sitios_ids = g_sitios['id_negocio'].unique()

# Filtrar las tablas por los IDs que coincidan con los de g_sitios
address = g_address[g_address['id_negocio'].isin(g_sitios_ids)]
horarios = g_horarios[g_horarios['id_negocio'].isin(g_sitios_ids)]
misc = g_misc[g_misc['id_negocio'].isin(g_sitios_ids)]
relative_r = g_relative_results[g_relative_results['id_negocio'].isin(g_sitios_ids)]

# Verificacion
print(f"Tamaño de la tabla address filtrada: {len(address)} filas")
print(f"Tamaño de la tabla horarios filtrada: {len(horarios)} filas")
print(f"Tamaño de la tabla horarios filtrada: {len(misc)} filas")
print(f"Tamaño de la tabla horarios filtrada: {len(relative_r)} filas")

Tamaño de la tabla address filtrada: 17376 filas
Tamaño de la tabla horarios filtrada: 17376 filas
Tamaño de la tabla horarios filtrada: 53333 filas
Tamaño de la tabla horarios filtrada: 69812 filas


In [10]:
# Guardamos las tablas nuevas actualizadas.
import os
os.makedirs('../../../data', exist_ok=True)
address.to_parquet('../../../data/g_address_v3.parquet', index=False)
horarios.to_parquet('../../../data/g_horarios_v3.parquet', index=False)
misc.to_parquet('../../../data/g_misc_v4.parquet', index=False)
relative_r.to_parquet('../../../data/g_relative_results_v3.parquet', index=False)
g_sitios.to_parquet('../../../data/g_sitios_v3.parquet', index=False)

tipos de datos de g_sitios corregir

In [3]:
import pandas as pd
g_sitios = pd.read_parquet(r'C:\Users\mauri\OneDrive\Escritorio\proyectogrupal\data\g_sitios_v3.parquet', engine='pyarrow')
g_sitios

,name,id_negocio,description,avg_rating,num_of_reviews,price,url,id_estado,id_category_general
0,"Seligman Fountain - ""Bear and Faun""",0x89c2f6163ac2864f:0xa4213a2f66c15d8f,None,4.5,4,None,https://www.google.com/maps/place//data=!4m2!3...,13,14.0
1,101 Mobility,0x864c1f10c079a921:0x95710cd26231cc3,None,5.0,45,None,https://www.google.com/maps/place//data=!4m2!3...,6,7.0
2,All Time Auto Transport,0x88d9050582e1e64b:0x33457ba08c64df20,None,4.8,96,None,https://www.google.com/maps/place//data=!4m2!3...,5,14.0
3,Scouts,0x88e783a0a141ae61:0xc8892c7a3b97364b,None,5.0,6,None,https://www.google.com/maps/place//data=!4m2!3...,5,7.0
4,Modoc County Record Inc,0x54cb90a9d85b5a37:0x2a6d31d462a28f2e,None,4.3,7,None,https://www.google.com/maps/place//data=!4m2!3...,2,14.0
...,...,...,...,...,...,...,...,...,...
17371,Al Kabab,0x8640dde02a02c4fb:0x6148638fa0f40f3b,None,3.8,125,None,https://www.google.com/maps/place//data=!4m2!3...,6,7.0
17372,LyondellBasell,0x8640bf3b561a0ed3:0x4eb4bef7ee8ee61,None,4.5,27,None,https://www.google.com/maps/place//data=!4m2!3...,6,7.0
17373,The Restaurant School at Walnut Hill College,0x89c6c6fa06c8981b:0x66ec244795b4378f,None,4.2,86,$$,https://www.google.com/maps/place//data=!4m2!3...,12,7.0
17374,Sebring Salt Spa,0x88dc59dedab9a643:0x9f2fdc4b800c3f2a,None,4.9,8,None,https://www.google.com/maps/place//data=!4m2!3...,5,7.0


In [4]:
g_sitios.price.value_counts()

price
$$      408
$       405
₩₩       53
$$$      42
₩        29
$$$$     11
₩₩₩       5
₩₩₩₩      1
Name: count, dtype: int64

Son todos signos que no dan informacion. Se borra

In [6]:
g_sitios = g_sitios.drop(columns=['price'])
g_sitios['id_category_general'] = g_sitios['id_category_general'].astype('int32')
g_sitios['id_estado'] = g_sitios['id_estado'].astype('int32')


In [8]:
g_sitios.to_parquet('../../../data/g_sitios_v4.parquet', index=False)